In [94]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time

## Download dataset and Tranformers library

In [95]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
# zalo data
# Đọc dữ liệu từ tệp tin JSON
with open('zac2022_train_merged_final.json', 'rb') as f:
    data = json.load(f)

zalo_data = data["data"]
zalo_traindata = []

for item in zalo_data:
  if item['category'] == "FULL_ANNOTATION":
    data_item = {}
    answer = {}
    data_item['context'] = item['text']
    data_item['question'] = item['question']
    answer['text'] = item['short_candidate']
    answer['answer_start'] = item['text'].find(item['short_candidate'])
    data_item['answer'] = answer
    zalo_traindata.append(data_item)

# print(zalo_traindata[0])
zalo_train = zalo_traindata[:4000]
zalo_test = zalo_traindata[4000:]

In [97]:
# facebook_data
# Đọc dữ liệu từ tệp tin JSON
with open('test-context-vi-question-en.json', 'rb') as f:
    data = json.load(f)

facebook_data = data["data"]
facebook_traindata = []

for para in facebook_data:
  for item in para['paragraphs']:
    data_item = {}
    answer = {}
    data_item['context'] = item['context']
    data_item['question'] = item['qas'][0]['question']
    answer['text'] = item['qas'][0]['answers'][0]['text']
    answer['answer_start'] = item['qas'][0]['answers'][0]['answer_start']
    data_item['answer'] = answer
    facebook_traindata.append(data_item)

facebook_train = facebook_traindata
#print(len(facebook_train))
#print(facebook_data[0]['paragraphs'])

# test
with open('dev-context-vi-question-en.json', 'rb') as f:
    data = json.load(f)

facebook_data = data["data"]
facebook_traindata = []

for para in facebook_data:
  for item in para['paragraphs']:
    data_item = {}
    answer = {}
    data_item['context'] = item['context']
    data_item['question'] = item['qas'][0]['question']
    answer['text'] = item['qas'][0]['answers'][0]['text']
    answer['answer_start'] = item['qas'][0]['answers'][0]['answer_start']
    data_item['answer'] = answer
    facebook_traindata.append(data_item)

facebook_test = facebook_traindata
#print(len(facebook_test))


In [98]:
# mailong_data
# Đọc dữ liệu từ tệp tin JSON
# train
with open('train-v2.0.json', 'rb') as f:
    data = json.load(f)

mailong_data = data["data"]
mailong_traindata = []

for para in mailong_data:
  for item in para['paragraphs']:

    answer = item['qas'][0]['answers']
    if answer != []: # check answer
      data_item = {}
      answer = {}
      data_item['context'] = item['context']
      data_item['question'] = item['qas'][0]['question']
      answer['text'] = item['qas'][0]['answers'][0]['text']
      answer['answer_start'] = item['qas'][0]['answers'][0]['answer_start']
      data_item['answer'] = answer
      mailong_traindata.append(data_item)

#print(mailong_traindata[2]['paragraphs'][0]['qas'])
mailong_train = mailong_traindata
print(len(mailong_train))

# test
with open('dev-v2.0.json', 'rb') as f:
    data = json.load(f)

mailong_data = data["data"]
mailong_traindata = []
for para in mailong_data:
  for item in para['paragraphs']:

    answer = item['qas'][0]['answers']
    if answer != []: # check answer
      data_item = {}
      answer = {}
      data_item['context'] = item['context']
      data_item['question'] = item['qas'][0]['question']
      answer['text'] = item['qas'][0]['answers'][0]['text']
      answer['answer_start'] = item['qas'][0]['answers'][0]['answer_start']
      data_item['answer'] = answer
      mailong_traindata.append(data_item)

mailong_test = mailong_traindata
print(len(mailong_test))

985
141


## ***Step 2:*** Merge the data

Here I take and store the texts, queries and answers from the train and validation .json files. If we look carefully we will see that in these files there are a number of queries and answers for each passage. I save these informations into lists.

In [103]:
texts = []
queries = []
answers = []

# Search for each passage, its question and its answer
for item in zalo_train:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

# Search for each passage, its question and its answer
for item in facebook_train:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

# Search for each passage, its question and its answer
for item in mailong_train:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

train_texts, train_queries, train_answers = texts, queries, answers

In [102]:
texts = []
queries = []
answers = []

# Search for each passage, its question and its answer
for item in zalo_test:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

# Search for each passage, its question and its answer
for item in facebook_test:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

# Search for each passage, its question and its answer
for item in mailong_test:
    # Store every passage, query and its answer to the lists
    texts.append(item['context'])
    queries.append(item['question'])
    answers.append(item['answer'])

val_texts, val_queries, val_answers = texts, queries, answers

## ***Step 3:*** Check the data

In [104]:
print(len(train_texts))
print(len(train_queries))
print(len(train_answers))

9780
9780
9780


In [105]:
print("Passage: ",train_texts[0])
print("Query: ",train_queries[0])
print("Answer: ",train_answers[0])

Passage:  có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...
Query:  Đất nước nào không có quân đội
Answer:  {'text': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...', 'answer_start': 53}


As you can see we have 20302 passages, queries and answers from the validation data

In [83]:
print(len(val_texts))
print(len(val_queries))
print(len(val_answers))

1581
1581
1581


In [106]:
print("Passage: ",val_texts[0])
print("Query: ",val_queries[0])
print("Answer: ",val_answers[0])

Passage:  Tháp Eiffel ( tiếng Pháp : Tour Eiffel ) là một công trình kiến trúc bằng thép nằm trên công viên Champ-de-Mars , cạnh sông Seine , thành phố Paris .
Query:  tháp eiffel toạ lạc ở công viên nào tại thành phố Paris
Answer:  {'text': 'công viên Champ-de-Mars', 'answer_start': 88}


This was helped me to split my dataset in order to run my notebook in my first attemps.

In [ ]:
# train_texts = train_texts[:10]
# train_queries = train_queries[:10]
# train_answers = train_answers[:10]

# val_texts = val_texts[:10]
# val_queries = val_queries[:10]
# val_answers = val_answers[:10]

In [ ]:
# print(len(train_texts))
# print(len(train_queries))
# print(len(train_answers))

In [ ]:
# print(len(val_texts))
# print(len(val_queries))
# print(len(val_answers))

## ***Step 4:*** Find the end position character

Because Bert model needs both start and end position characters of the answer, I have to find it and store it for later. Sometimes, I notice that SQuAD anwers "eat" one or two characters from the real answer in the passage. For example, (as a colleague said in Piazza) for the word "sixth" in passage, SQuAD give the answer of "six". So in these cases I select to handle this problem by "cutting" the passage by 1 or 2 characters to be the same as the given answer. This strategy is because BERT works with ***tokens*** of a specific format so I needed to process the squad dataset to keep up with the input that BERT is waiting for.

Find end position character in train data

In [107]:
for answer, text in zip(train_answers, train_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

Find end position character in validation data

In [108]:
for answer, text in zip(val_answers, val_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

## ***Step 5:*** Tokenize passages and queries

In this task is asked to select the BERT-base pretrained model “bert-base-uncased” for the tokenization

In [109]:
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

## ***Step 6:*** Convert the start-end positions to tokens start-end positions

In [ ]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []

  count = 0

  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length

    # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
    if end_positions[-1] is None:
      end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - 1)
      # if end position is still None the answer passage has been truncated
      if end_positions[-1] is None:
        count += 1
        end_positions[-1] = tokenizer.model_max_length

  print(count)

  # Update the data in dictionary
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

I observed that after tokenize the end position character, sometimes was still None. This happended only for 10 answers in train data (of total 86821) and 16 answers in validation data (of total 20302). So I decided to move the answer 1 position left. If it was still None then I give them the model_max_length as before. I have to refer that I was trying to see if the answers in this case was 1 postition after (so I added +1 to the end position) or 2 positions left or right (+/- 2 positions), but the answers that there are still None was more (ie 526, while with this code there only 10). So I kept this strategy in the end, in order to have as less as possible "burned" answers.

## ***Step 7:*** Create a Dataset class

Create a Squatdataset class (inherits from torch.utils.data.Dataset), that helped me to train and validate my previous data more easily and convert encodings to datasets.

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

## ***Step 8:*** Use of DataLoader

I put my previous data to DataLoader, so as to split them in "pieces" of 8 batch size. I will explain the selection of this value of batch size later.

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## ***Step 9:*** Use GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available()
                      else 'cpu')

## ***Step 10:*** Build the Bert model

I select BertForQuestionAnswering from transformers library, as it was the most relative with this task. When we instantiate a model with from_pretrained(), the model configuration and pre-trained weights of the specified model are used to initialize the model. Moreover, I used the PyTorch optimizer of AdamW which implements gradient bias correction as well as weight decay.



In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-base-multilingual-uncased').to(device)

optim = AdamW(model.parameters(), lr=5e-5)
# optim = AdamW(model.parameters(), lr=3e-5)
# optim = AdamW(model.parameters(), lr=2e-5)

# epochs = 2
epochs = 3
# epochs = 4

This message is a warning that I should fine tune my model before I test it, in order to have a good performance.

## ***Step 11:*** Train and Evaluate Model

Training of model was done exactly as in the previous projects.

In [ ]:
whole_train_eval_time = time.time()

train_losses = []
val_losses = []

print_every = 1000

for epoch in range(epochs):
  epoch_time = time.time()

  # Set model in train mode
  model.train()

  loss_of_epoch = 0

  print("############Train############")

  for batch_idx,batch in enumerate(train_loader):

    optim.zero_grad()

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    # do a backwards pass
    loss.backward()
    # update the weights
    optim.step()
    # Find the total loss
    loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
      print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(train_loader)
  train_losses.append(loss_of_epoch)

  ##########Evaluation##################

  # Set model in evaluation mode
  model.eval()

  print("############Evaluate############")

  loss_of_epoch = 0

  for batch_idx,batch in enumerate(val_loader):

    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss = outputs[0]
      # Find the total loss
      loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
       print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(val_loader)
  val_losses.append(loss_of_epoch)

  # Print each epoch's time and train/val loss
  print("\n-------Epoch ", epoch+1,
        "-------"
        "\nTraining Loss:", train_losses[-1],
        "\nValidation Loss:", val_losses[-1],
        "\nTime: ",(time.time() - epoch_time),
        "\n-----------------------",
        "\n\n")

print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))

In [ ]:
# Save model
torch.save(model,"/content/drive/MyDrive/finetunedmodel")

## ***Step 12:*** Plot train and validation losses

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(15,10))

ax.set_title("Train and Validation Losses",size=20)
ax.set_ylabel('Loss', fontsize = 20)
ax.set_xlabel('Epochs', fontsize = 25)
_=ax.plot(train_losses)
_=ax.plot(val_losses)
_=ax.legend(('Train','Val'),loc='upper right')

Evaluate

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')

# Load the fine-tuned modeol
model = torch.load("/content/drive/MyDrive/finetunedmodel",map_location=torch.device('cpu'))
model.eval()

In [ ]:
def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)


In [ ]:
def give_an_answer(context,query,answer):

  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print("\n")


In [ ]:
context = ["có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...",
           "Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , và Alexandria trở thành thủ phủ của tỉnh Ai Cập trong đế quốc La Mã .",
           "Đoạn trường tân thanh , thường được biết đến đơn giản là Truyện Kiều , là một truyện thơ của thi sĩ Nguyễn Du . Đây được xem là truyện thơ nổi tiếng nhất và xét vào hàng kinh điển trong văn học Việt Nam , tác phẩm được viết bằng chữ Nôm theo thể lục bát , gồm 3254 câu .",
           "Ngày 3 tháng 10 năm 2018, Nguyễn Phú Trọng được Ban Chấp hành Trung ương Đảng Cộng sản Việt Nam thống nhất giới thiệu Quốc hội bầu làm Chủ tịch nước. Sau khi được Quốc hội Việt Nam khóa 14 bầu làm Chủ tịch nước vào ngày 23 tháng 10 năm 2018, ông là chính khách thứ ba trong lịch sử Đảng Cộng sản Việt Nam nắm giữ cả hai chức vụ đứng đầu Đảng Cộng sản cầm quyền và Nhà nước, sau Hồ Chí Minh và Trường Chinh.",
           "Đỉnh núi nằm ở phần trung tâm của dãy núi Đại Kavkaz , phía đông nam của núi Elbrus , ngọn núi cao nhất của châu Âu ."
          ]

queries = ["Đất nước nào không có quân đội",
           "Thành phố nào là thủ phủ của Ai Cập trong đế quốc La Mã",
           "Truyện Kiều được viết theo thể nào",
           "chủ tịch nước việt nam là ai",
           "Núi nào cao nhất châu âu"
          ]
answers = ["Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...",
           "Alexandria",
           "theo thể lục bát",
           "Nguyễn Phú Trọng",
           "núi Elbrus"
          ]

for c,q,a in zip(context,queries,answers):
  give_an_answer(c,q,a)